## Cofi Reader: A full history of Bloomberg Company Financials
- Global company examples (id_bb_company) in this testing notebook: ['101743','13561981', '101534', '111680', '118412', '100232', '115114', '115709', '111100' , '179551', '11092218']

In [42]:
%%capture
!pip install --upgrade bloomberg.es.lacus.location
!pip install matplotlib

In [43]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pyarrow.dataset as ds
import pyspark.sql.functions as F

from bloomberg.es.lacus.location import LacusLocation

pd.set_option("max_columns", None)
pd.set_option("max_rows", 500)

In [44]:
def setup_dsp_pyspark(
    env="dev",
    driver_memory="16g",
    driver_cores=4,
    driver_max_result_size="16g",
    executor_memory="16g",
    executor_cores=5,
    num_executors=5,
    app_name="lqa-jupyter",
    datastore_version="0.1.23",
    iceberg_runtime_version="0.11.0",
    iceberg_version="1.0.24",
    write_to_dev=True,
):
    from pyspark import SparkConf, SparkContext
    from pyspark.sql import SQLContext, SparkSession

    SparkSession.builder.config(
        "spark.jars",
        "http://artprod.dev.bloomberg.com/artifactory/scala-repos/com/bloomberg/raap/datastore-python_2.11/{0}-RELEASE/datastore-python_2.11-{0}-RELEASE-assembly.jar,http://artifactory.inf.bloomberg.com/artifactory/root-repos/org/apache/iceberg/iceberg-spark-runtime/{1}/iceberg-spark-runtime-{1}.jar,http://artifactory.inf.bloomberg.com/artifactory/root-repos/com/bloomberg/lacus/iceberg/{2}/iceberg-{2}.jar".format(
            datastore_version, iceberg_runtime_version, iceberg_version
        ),
    )
    SparkSession.builder.config(
        "spark.driver.extraJavaOptions",
        "-Ddatastore.location=/groups/lqa/datastore -Denv.id=dev -Djava.net.preferIPv4Stack=true -DHADOOP_USER_NAME=lqa -Dhive.metastore.uris=thrift://es-presto-meta-{env}.bdns.bloomberg.com:9083 -Dlacus.iceberg.useWarehouseDirectory=true".format(
            env=env
        ),
    )
    SparkSession.builder.config(
        "spark.executor.extraJavaOptions",
        "-Ddatastore.location=/groups/lqa/datastore -Denv.id=dev -DHADOOP_USER_NAME=lqa -DHADOOP_CONF_DIR=/job/.local/lqa_hadoop_conf",
    )
    SparkSession.builder.config("spark.driver.memory", driver_memory)
    SparkSession.builder.config("spark.executor.memory", executor_memory)
    SparkSession.builder.config("spark.executor.cores", str(executor_cores))
    SparkSession.builder.config("spark.executor.instances", str(num_executors))
    SparkSession.builder.config("spark.driver.maxResultSize", str(driver_max_result_size))
    SparkSession.builder.config("spark.driver.cores", str(driver_cores))
    SparkSession.builder.config("spark.app.name", app_name)
    SparkSession.builder.config(
        "spark.hadoop.fs.s3a.endpoint", "http://s3.prod.obdc.bcs.bloomberg.com"
    )

    # To write to dev
    if write_to_dev:
        SparkSession.builder.config(
            "spark.hadoop.fs.s3a.bucket.public-iceberg-dev.endpoint", "http://s3.dev.obdc.bcs.bloomberg.com"
        )
        SparkSession.builder.config(
            "spark.sql.warehouse.dir", "s3a://public-iceberg-dev/"
        )

    sess = SparkSession.builder.enableHiveSupport().getOrCreate()
    return sess

In [45]:
spark = setup_dsp_pyspark()
spark

In [46]:
#spark.stop()

##### User inputs

In [47]:
company = 101534  # Select a companies id_bb_compamny from list above
item = "bs_tot_cap"  # Select item from cofiSlim list
period = "%A%"  # Select period (A for Annual, Q for namr interims & S for elsewehre interims)

In [73]:
# select cofi item from list above
cols = [
    
    "id_bb_company",
    "long_comp_name",
    "eqy_fund_year",
    "latest_period_end_dt_full_record",
    "fund_per",
    "rank_num",
    "sequence_num",
    "fundamental_public_date",
    "dl_snapshot_date_source",
    "eqy_fund_crncy",
    item
]

# Only run these if you are pulling all period types
#cofiSlim["fund_per"] = pd.Categorical(cofiSlim["fund_per"], ["Q1", "Q2", "Q3", "Q4", "S1", "S2", "A"])

drops = [
    
    "id_bb_company",
    "long_comp_name",
    "eqy_fund_year",
    "fund_per",
    "eqy_fund_crncy",
    "latest_period_end_dt_full_record",
    item
]

pit = [
    
    "id_bb_company",
    "long_comp_name",
    "eqy_fund_year",
    "fund_per",
    "latest_period_end_dt_full_record",
    "fundamental_public_date",
    "eqy_fund_crncy"
]

latest = [
    
    "id_bb_company",
    "long_comp_name",
    "eqy_fund_year",
    "fund_per",
    "latest_period_end_dt_full_record",
    "eqy_fund_crncy",
    
]

# Sample List of companies
companies = (
    101743, # MSFT US Equity
    13561981, # V US Equity
    101534, # ULVR LN Equity
    111680, # DGE LN Equity
    118412, # SAP GY Equity
    100232, # BHP AT Equity
    115114, #7203 JT Equity (Toyota)
    115709, # DBS SP Equity
    111100, # ANTO LN Equity
    179551, # CNI US Equity
    11092218, # FB US Equity
    69466781, # COIN US Equity
    130891 # 000002 CH Equity (CHINA VANKE CO LTD -A)
)
filters = [f"id_bb_company in {companies}"]

# Dates for pull (current & hist)
sm_st = "2020-01-01"
sm_end = "2021-01-01"

st_dt = "1980-01-01"
end_dt = "2021-04-01"

hist_st_dt = "1980-01-01"
hist_end_dt = "2019-09-30"

items = [
    "bs_other_intangible_assets",
    "bs_cash_near_cash_item",
    "bs_acct_note_rcv",
    "bs_inventories",
    "bs_cur_asset_report",
    "bs_net_fix_asset",
    "bs_tot_asset",
    "bs_acct_payable",
    "bs_st_borrow",
    "bs_cur_liab",
    "bs_lt_borrow",
    "bs_pfd_eqty_and_hybrid_cptl",
    "minority_noncontrolling_interest",
    "bs_sh_out",
    "bs_curr_portion_lt_debt",
    "bs_tot_liab2",
    "bs_tot_cap",
    "total_equity",
    "tot_common_eqy",
    "tang_book_val_per_sh",
    "tot_debt_to_tot_asset",
    "cur_ratio",
    "working_capital",
    "net_debt",
    "short_and_long_term_debt",
    "tot_debt_to_tot_eqy",
    "px_to_book_ratio",
    "bs_num_of_tsy_sh",
    "cash_flow_per_sh",
    "px_to_cash_flow",
    "ev_to_t12m_ebitda",
    "earn_yld",
    "revenue_per_sh",
    "return_com_eqy",
    "return_on_inv_capital",
    "return_on_asset",
    "pe_ratio",
    "px_to_sales_ratio",
    "cf_net_inc",
    "cf_depr_amort",
    "cf_cash_from_oper",
    "cf_cap_expend_prpty_add",
    "cf_cash_from_inv_act",
    "cf_dvd_paid",
    "cf_cash_from_fnc_act",
    "cf_decr_lt_borrow",
    "cf_depr_exp",
    "cf_nt_csh_rcvd_pd_for_acquis_div",
    "chng_work_cap",
    "cf_free_cash_flow",
    "free_cash_flow_yield",
    "sales_rev_turn",
    "is_cogs_to_fe_and_pp_and_g",
    "is_operating_expn",
    "is_oper_inc",
    "is_int_expense",
    "is_inc_tax_exp",
    "min_noncontrol_interest_credits",
    "net_income",
    "is_eps",
    "is_diluted_eps",
    "is_earn_bef_xo_items_per_sh",
    "is_cap_int_exp",
    "is_basic_eps_cont_ops",
    "is_dil_eps_cont_ops",
    "is_dil_eps_bef_xo",
    "ebit",
    "ebitda",
    "eqy_dps",
    "gross_margin",
    "interest_coverage_ratio",
    "net_rev",
    "prof_margin",
    "ebitda_to_revenue",
    "is_avg_num_sh_for_eps",
]

In [49]:
cofi_slim = LacusLocation(data_name="cofi_slim_20210625",db_name="qrp",catalog="icebergdev").get(df_type="spark")

In [50]:
# use this to clear incorrect credential (toolkit) entry, ie reset the credential requirements and try again

'''import os
os.environ["PRESTO_USER"] = ""
os.environ["PRESTO_PASSWORD"] = ""'''

'import os\nos.environ["PRESTO_USER"] = ""\nos.environ["PRESTO_PASSWORD"] = ""'

##### Latest

In [66]:
cofi_slim.select(*cols) \
    .filter(cofi_slim["id_bb_company"] == company) \
    .filter(cofi_slim.fund_per.like(period)) \
    .dropna(subset=[item]) \
    .dropDuplicates(drops) \
    .orderBy("eqy_fund_year", "fund_per", "rank_num", "sequence_num", ascending=False) \
    .dropDuplicates(latest) \
    .orderBy("eqy_fund_year", ascending=False) \
    .show(200)

+-------------+--------------+-------------+--------------------------------+--------+--------+------------+-----------------------+-----------------------+--------------+----------+
|id_bb_company|long_comp_name|eqy_fund_year|latest_period_end_dt_full_record|fund_per|rank_num|sequence_num|fundamental_public_date|dl_snapshot_date_source|eqy_fund_crncy|bs_tot_cap|
+-------------+--------------+-------------+--------------------------------+--------+--------+------------+-----------------------+-----------------------+--------------+----------+
|       101534|  Unilever PLC|         2020|                      2020-12-31|       A|       3|          17|             2021-03-10|             2021-03-11|           EUR|   44902.0|
|       101534|  Unilever PLC|         2019|                      2019-12-31|       A|       3|          23|             2020-03-09|             2020-03-10|           EUR|   41761.0|
|       101534|  Unilever PLC|         2018|                      2018-12-31|       A

##### Point in time

In [74]:
cofi_slim.select(*cols) \
    .filter(cofi_slim["id_bb_company"] == company) \
    .filter(cofi_slim.fund_per.like(period)) \
    .dropna(subset=[item]) \
    .dropDuplicates(drops) \
    .orderBy("eqy_fund_year", "fund_per", "rank_num", "sequence_num", ascending=False) \
    .dropDuplicates(pit) \
    .orderBy("eqy_fund_year", "fund_per", "rank_num", "sequence_num", ascending=False) \
    .show(200)

+-------------+--------------+-------------+--------------------------------+--------+--------+------------+-----------------------+-----------------------+--------------+----------+
|id_bb_company|long_comp_name|eqy_fund_year|latest_period_end_dt_full_record|fund_per|rank_num|sequence_num|fundamental_public_date|dl_snapshot_date_source|eqy_fund_crncy|bs_tot_cap|
+-------------+--------------+-------------+--------------------------------+--------+--------+------------+-----------------------+-----------------------+--------------+----------+
|       101534|  Unilever PLC|         2020|                      2020-12-31|       A|       3|          17|             2021-03-10|             2021-03-11|           EUR|   44902.0|
|       101534|  Unilever PLC|         2020|                      2020-12-31|       A|       2|           3|             2021-02-04|             2021-02-04|           EUR|   44960.0|
|       101534|  Unilever PLC|         2019|                      2019-12-31|       A